In [0]:
Hyperparamters : 
    Where? before you register to UC.
    what? -- control the learning process by using different parameter values, significantly affecting the model's performance.
    how? -- 1. grid search 2. random search 3. Bayesian optimization

    hyperparmater: Hyperopt, Optuna

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score

In [0]:
df=pd.read_csv("/Volumes/ml/default/raw/heart.csv")

In [0]:
df.head()

In [0]:
df.shape

In [0]:
X=df.iloc[:,0:-1]
y=df.iloc[:,-1]

In [0]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [0]:
rf=RandomForestClassifier()
gb=GradientBoostingClassifier()
svc=SVC()
lr=LogisticRegression()

In [0]:
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred=rf.predict(X_test)
accuracy_score(y_test,y_pred)

In [0]:
svc.fit(X_train,y_train)
y_pred=svc.predict(X_test)
accuracy_score(y_test,y_pred)

In [0]:
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred)

In [0]:
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
accuracy_score(y_test,y_pred)

In [0]:
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred=rf.predict(X_test)
accuracy_score(y_test,y_pred)

In [0]:
rf=RandomForestClassifier(max_samples=0.75,random_state=42)
rf.fit(X_train,y_train)
y_pred=rf.predict(X_test)
accuracy_score(y_test,y_pred)

In [0]:
from sklearn.model_selection import cross_val_score
np.mean(cross_val_score(RandomForestClassifier(max_samples=0.75),X,y,cv=5,scoring='accuracy'))

### Grid Search

In [0]:
#number of trees in random forest
n_estimators=[20,60,100,120]

#number of features to consider at every split
max_features=[0.2,0.6,1.0]

#maximum number of levels in tree
max_depth=[2,8,None]

#number of samples 
max_samples=[0.5,0.75,1.0]


# total combination
#4 *3 *3 *3 = 108



In [0]:
param_grid={'n_estimators':n_estimators,
            'max_features':max_features,
            'max_depth':max_depth,
            'max_samples':max_samples
}
print(param_grid)

In [0]:
rf=RandomForestClassifier()

In [0]:
from sklearn.model_selection import GridSearchCV
grid=GridSearchCV(estimator=rf,
                  param_grid=param_grid,
                  cv=5,
                  verbose=2,
                  n_jobs=-1)

In [0]:
grid.fit(X_train,y_train)

In [0]:
grid.best_params_

In [0]:
grid.best_score_

### Random Search

In [0]:
# Number of trees in random forest
n_estimators = [20,60,100,120]

# Number of features to consider at every split
max_features = [0.2,0.6,1.0]

# Maximum number of levels in tree
max_depth = [2,8,None]

# Number of samples
max_samples = [0.5,0.75,1.0]

# Bootstrap samples
bootstrap = [True,False]

# Minimum number of samples required to split a node
min_samples_split = [2, 5]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]

In [0]:
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
              'max_samples':max_samples,
              'bootstrap':bootstrap,
              'min_samples_split':min_samples_split,
              'min_samples_leaf':min_samples_leaf
             }
print(param_grid)

In [0]:
from sklearn.model_selection import RandomizedSearchCV

rf_grid = RandomizedSearchCV(estimator = rf, 
                       param_distributions = param_grid, 
                       cv = 5, 
                       verbose=2, 
                       n_jobs = -1)

In [0]:
rf_grid.fit(X_train,y_train)

In [0]:
rf_grid.best_params_

In [0]:
rf_grid.best_score_

In [0]:
%pip install optuna
%pip install optuna-integration # Integration with MLflow

In [0]:
dbutils.library.restartPython()

In [0]:
import optuna
from sklearn.model_selection import cross_val_score

#define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    #create the RandomForestClassifier 
    model=RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    #perform 3-fold cross-validation
    score=cross_val_score(model,X_train,y_train,cv=3,scoring='accuracy').mean()

    return score


In [0]:
#create a study object 
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

In [0]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')